# Monte Carlo Tree Search

ISAE 2016 / 2017 - Tic Tac Toe Tutorial

*Nicolas Schneider (email: <nls.schneider@gmail.com>)*


## Introduction

Monte Carlo tree search is a decision process used in combinatorial problems. 

In game playing such tic tac toe, chess, go, hex … setting up an artificial intelligence player 
relies on a graph in which all different outcomes could be computed (ie: fully deterministic). 

Many heuristics and algorithms have been developed to provide the optimal move according to a specific game state. 
MinMax or Alpha Beta approaches show good results but the whole combinatory problem should be model.  

The combinatory of some games is so huge that it becomes impossible to model all the different states.
The combinatory for a 9x9 Go games board is approximately 5E120. 

Therefore heuristic approaches should be used to reduce the search space and favor accurate moves.

MCTS becomes famous by beating world best players in a Go game. 
That is particularly significant as the whole AI community struggle to develop an AI player that 
could compete with amateur players.

## MCTS Concepts

The basic of MCTS is simple:  

   > **A search tree is built, node by node, according to the outcomes of simulated playouts**. 

* **Nodes** represent game states  
* **Arcs** represent possible actions from a game state to another one. 

Initially, each node stores two values:  

* **Play value**: that indicates how many times the node has been played.  
* **Win value**:  that indicates how many times the node has won the game (or a quantitative payoff value).


The process could be divided in 4 different steps: 
1. Selection
2. Expansion
3. Simulation
4. Back Propagation.


<img src="images/MCTS.png" height="70%" width="70%">

The whole process is repeated n times

### Selection

The selection process is the heart of the MCTS approach. Somtime called *Tree Policy* the selection steps has two crucial roles:

> * **Select which action to play.**
> * **Define the topology of the search space by selecting nodes to explore.**

Starting from the current state **r** (depending on the game state), we select most promising nodes, until we reach a still non fully exploring node or a leaf.

   *But, how to select promising nodes ?*  

Each node has **play** and **win** values. These values are used to compute a **gain** for each possible action.

Many strategies exist, but we will review here only the best one: **Upper Confidence Bound applied to Trees** (UCB).

The UCB strategy looking to maximize the estimate rewards for each node and has the particularity to minimize your “regret” by choosing an action instead of another one. UCB formula is typically this one:


$$UCB1 = \frac{w_i}{n_i} + \gamma . \sqrt{\frac{\ln n}{n_i}}$$

where:  
* $w_i$ is the reward of node $i$ (ie. the **win** value of node $i$)  
* $n_i$ is the number of time node $i$ was played (ie. the **play** value of node $i$)  
* $\gamma$ is the exploration parameter (ie. typically equal to $\sqrt{2}$)  
* $n$ is the overall amount of play so far (ie. the sum of play value for all sons or play value of the parent)  

To be noted that $\frac{w_i}{n_i}$ encourages the exploration of higher reward choices, while $\sqrt{\frac{\ln n}{n_i}}$ encourages the exploration of less visited choices.

## Expansion

> **The expansion step aims to select a starting point for the Monte Carlo simulation.**

Once we have selected a node and if that node is not a terminal node (ie: end of the game), we select randomly a child **c**. This child **c** will be our starting point to perform a Monte Carlo simulation during the next step.


## Simulation

> **The simulation step aims to compute a payoff.**

We run a simulation from the node expand during the expansion step (**c**) until a payoff is collected (ie. or limited to a fix nuber of simulation step). 

The selection process during the simulation is made randomly. Once the end of the game is reached, we compute a reward depending if the player won or loose the game. With that payoff, we could move to the step 4: the back propagation.


## Back Propagation

> **The back propagation step aims to update the selected nodes of the graph build so far.**

We update all visited nodes from the one we have expand in the expansion step **c** to the starting node **r** where we started the selection step. 

* The **play** value is increment by 1
* The **win** value is increment by 1 (or the payoff value) for nodes that refer to the winner


## Benefits

* MCTS do not require any expert knowledge. MCTS only need to know next states from a given condition using an action and game’s end conditions. By consequence, MCTS is generic and could be used with very minor modifications to many different games and problems.


* MCTS builds asymmetric search tree and adapts its topology to the search space. The exploration / exploitation paradigm is directly take in account in the selection process and allows to spent time in the more relevant part of the tree.


* MCTS is an online algorithm that provides decisions according to non-predictive user moves. 


* MCTS is adaptable and could be stop at any time. The search tree built by the algorithm could be stored and reuse later.


* Finally, MCTS is very simple (not simplistic) and easy to code.


# Example on Tic-Tac-Toe

We will illustrate the MCTS algorithm on a tic tac toe board game. A 3x3 board game in which you should align 3 crosses or 3 circles to win the game.

The game is not particulary complex but without optimisation (ie: symetrie, ...) we have $9!=362880$ possible sequence (*(state, action)*) and $3^9 = 19683$ possible states.

Therefore, we want to demonstrate that:

1. MCTS is a reinforcment learning algorithm and our AI player should learn and improve itself
2. MCTS is a powerfull algorithm to select the best strategy without looking for all solution (without building the whole graph)

First, we will implement our MCTS algorithm from a generic manner. The objective is to be able to reuse our MCTS for any board game without many modifications.

## MCTS implementation

We seen above that MCTS is made of 4 functions + a 5th to made the loop

1. selection
2. expansion
3. simulation
4. back_propagation
5. play


The MCTS algorithm should have to manipulate some objects of the board game. Therefore we assess that the board game has the following functions:

description | function | return
----|----|-----
the current player | joueur(state) | the current player (1 or 2)
the available moves | available_move(state) | a list of actions
the next state | next_state(state, action, current_player) | the next state according to action and depending on the current_player
a player win | win(board, player) | true or false
the reward | payoff(current_player, state) | the reward for the current player corresponding to the state (-1=running, 1=player1, 2=player2, 0=nul)
the play function that actually make an action | play(state, action) | update player and board state

To initiate our MCTS we need few variables:

description | variable | type
------------|----------|------
a game | jeu | board game implementation
who is playing | joueur | id
the state list of the game | states | [jeu.board]
the current state of the game| current_state | jeu.board
the graph | graphe | {jeu.board:[]} #{key=jeu.board, value=[(action, state)]} (ie. a node with the list of sons)
the **win** values | wins | {jeu.board:0}
the **play** values | plays | {jeu.board:0}

The **selection** function should return the selected node and the list of visited nodes (we will need of that list to do the back propagation)

The **expansion** function should return the new expanded node and the update list of visited nodes

The **simulation** do the monte carlo simulation and return a payoff

The **back_propagation** function do not return any object but update wins and plays values

The **play** function made the loop n times and return an action

In [ ]:
from random import randint
import math


class mcts3:
    def __init__(self, jeu, playerIA):
        self.joueur = playerIA
        self.jeu = jeu
        self.states = [jeu.board]  # [état, état), ...]
        self.current_state = jeu.board  # état
        self.graphe = {jeu.board: []}  # {état : [(action, état), (action, état), ...]}
        self.wins = {jeu.board: 0}  # {état : nb_de_fois_gagné}
        self.plays = {jeu.board: 0}  # {état : nb_de_fois_visité}
        
    # return current_simulated_state, visited_state # s, [s, s, s, ...]
    def selection(self, current_simulated_state, visited_state):
        fils_nodes = self.graphe.get(current_simulated_state, 0)  # [s, s, s, ...]
        # critère d'arret ... pas de fils. On a finit la selection, on garde l'état courant
        while fils_nodes != 0 and len(fils_nodes) != 0:
            visited_state.append(current_simulated_state)

            # S'il pourait y avoir d'autre fils à explorer: politique d'exploration
            moves = self.jeu.available_move(current_simulated_state)  # mouvements autorisés
            if len(moves) != 0:  # il existe des fils non visités
                if randint(0, 10) <= 3:  # proba 0.2 d'explorer d'autre fils
                    # on garde le noeud courant comme selection à partir duquel on va faire l'expansion
                    return current_simulated_state, visited_state  # s, [s, s, s, ...]

            # on selectionne selon la formule UCB1 qui minimise la deception
            len_total = math.log(sum(self.plays[s[1]] for s in fils_nodes))
            max_value = -99999
            best_node = ()
            for s in fils_nodes:
                UCB1_value = (self.wins[s[1]] / self.plays[s[1]]) + math.sqrt(2) * math.sqrt(len_total / self.plays[s[1]])
                if UCB1_value >= max_value:
                    max_value = UCB1_value
                    best_node = s[1]

            current_simulated_state = best_node
            fils_nodes = self.graphe.get(current_simulated_state, 0)
        return current_simulated_state, visited_state  # s, [s, s, s, ...]
    
    
    def expension(self, current_simulated_state, visited_state):
        current_player = self.jeu.joueur(current_simulated_state)
        moves = self.jeu.available_move(current_simulated_state)  # mouvements autorisés
        fils_nodes = self.graphe.get(current_simulated_state, 0)  # [(a,s), (a,s), (a,s), ...]
        # cas d'exploration: a été choisi un noeud ou toutes les actions n'ont pas été explorées
        if (fils_nodes != 0):
            for fils_node in fils_nodes:
                moves.remove(fils_node[0])  # on retire les actions déjà visitées (ie: dont le fils existe)
        etat = current_simulated_state
        if (len(moves) != 0):
            # aleatoirement on choisit une action et developpons un nouvel état
            action = moves[randint(0, len(moves) - 1)]
            etat = self.jeu.next_state(current_simulated_state, action, current_player)
            visited_state.append(etat)
            # que l'on ajoute à la liste des états avec des stats egal à 0
            self.states.append(etat)
            # et mettons à jour le graphe
            fils = self.graphe.get(current_simulated_state, [])
            self.graphe[current_simulated_state] = fils + [(action, etat)]  # on enregistre le fils dans le graphe
            self.graphe[etat] = []
            self.wins[etat] = 0
            self.plays[etat] = 0
        return etat, visited_state
    
    
    # return payoff
    def simulation(self, current_simulated_state):
        current_player = self.jeu.joueur(current_simulated_state)
        payoff = self.jeu.payoff(current_player, current_simulated_state)
        # Monte carlo simulation, On choisit aleatoirement des actions jusqu'au critère d'arret: fin du jeu
        while(payoff == -1):
            moves = self.jeu.available_move(current_simulated_state)  # mouvements autorisés
            if (len(moves)!=0):
                # aleatoirement on choisit une action et developpons un nouvel état
                action = moves[randint(0, len(moves) - 1)]
                etat = self.jeu.next_state(current_simulated_state, action, current_player)
                # mise à jour de l'état courant
                current_simulated_state = etat
            current_player = self.jeu.joueur(current_simulated_state)
            payoff = self.jeu.payoff(current_player, current_simulated_state)
        return payoff
    
    
    def back_propagation(self, visited_state, payoff):
        for state in visited_state:
            # mise a jour de la valeur play
            playsState = self.plays.get(state, 0)
            self.plays[state] = playsState + 1
            # mise a jour de la valeur win en function du joueur
            if self.jeu.joueur(state) != payoff and payoff != 0: # on recompense les états qui gagne
                winsState = self.wins.get(state, 0)
                self.wins[state] = winsState + 1
            if self.jeu.joueur(state) == payoff and payoff != 0: # on puni les états qui perde
                winsState = self.wins.get(state, 0)
                self.wins[state] = winsState + 0 
            if payoff == 0:
                winsState = self.wins.get(state, 0)
                self.wins[state] = winsState + 0.5
                
                
    # return action
    def play(self, current_state, n):
        self.current_state = current_state
        move = 0
        # on simule n partie (ie. on construit le graphe sur n partie)
        while (move < n):
            current_simulated_state = current_state
            visited_state = []
            #current_player = self.jeu.joueur(current_state)
            
            #while (self.jeu.payoff(current_player, current_simulated_state) == -1):
            current_simulated_state, visited_state = self.selection(current_simulated_state, visited_state)
            current_simulated_state, visited_state = self.expension(current_simulated_state, visited_state)
            payoff = self.simulation(current_simulated_state)
            self.back_propagation(visited_state, payoff)
            #current_player = self.jeu.joueur(current_simulated_state)
                
            move += 1
        potential_play = self.graphe[current_state]
        max_value = -99999
        best_action = 0
        for (a, s) in potential_play:
            value = (self.wins[s] / self.plays[s])
            print("Action:", a, "\twins,plays:", self.wins[s], "/", self.plays[s], "\tvalue: %.3f" % value, "\tnb Etat:", len(self.graphe.keys()))
            if value >= max_value:
                max_value = value
                best_action = a
        return best_action

## The board game

This is a quite simple implementation. The board is a tuple of size 9 where each action refers to a position in the tuple. We store the status of the current player and who won the game.

We see above the function that we have to developp ... Lets do that!!

The main class and objects:

* joueur(state) 
* available_move(state)
* next_state(state, action, current_player)
* win(board, player)
* payoff(current_player, state) 
* play(state, action)

In [ ]:
class oxo:
    def __init__(self):
        self.current_player = 1
        self.actions = [0, 1, 2, 3, 4, 5, 6, 7, 8]
        self.board = (0,0,0,0,0,0,0,0,0)
        self.nb_move = 0
        self.end_game = 0 # -1: running, O: nul, 1: player 1 win, 2: player 2 win
        
    def joueur(self, board):
        J1=0
        J2=0
        for i in board:
            if i==1: J1+=1 
            if i==2: J2+=1 
        if J1==J2: return 1
        return(2)
    
    def available_move(self, state):
        am = []
        i = 0;
        for x in state:
            if x == 0: 
                am += [i]
            i += 1
        return am
    
    def next_state(self, state, action, player):
        stateList = list(state)
        stateList[action] = player
        return tuple(stateList)
    
    
    def win(self, b, p):
        if (b[0] == b[1] == b[2] == p or b[3] == b[4] == b[5] == p or b[6] == b[7] == b[8] == p or
            b[0] == b[3] == b[6] == p or b[1] == b[4] == b[7] == p or b[2] == b[5] == b[8] == p or
            b[0] == b[4] == b[8] == p or b[2] == b[4] == b[6] == p):
            return True
        else: return False
        
        
    def asWin(self):
        p = self.current_player
        b = self.board
        if (b[0] == b[1] == b[2] == p or b[3] == b[4] == b[5] == p or b[6] == b[7] == b[8] == p or
            b[0] == b[3] == b[6] == p or b[1] == b[4] == b[7] == p or b[2] == b[5] == b[8] == p or
            b[0] == b[4] == b[8] == p or b[2] == b[4] == b[6] == p):
            return True
        else: return False
        
    # -1: running, 0; execo, 1 joueur 1, 2 joueur 2    
    def payoff(self, p, b):
        nb_move = 0
        for i in b:
            if i != 0: nb_move += 1

        if self.win(b, 1):    return 1
        if self.win(b, 2):    return 2    
        if nb_move == 9: return 0        
        return -1
    
    
    def play(self, state, action):
        self.current_player = self.joueur(state)
        stateList = list(self.board)
        stateList[action] = self.current_player
        self.board = tuple(stateList)
        self.nb_move += 1
        #print("nb move: ", self.nb_move)
        self.actions.remove(action)
        
        
    def myPrint(self):
        b = []
        for x in self.board:
            if x == 1: b.append('X')
            else: 
                if x == 2: b.append('O')
                else: b.append('.')
        print()
        print("     ", b[0] , "  " , b[1] , "  " , b[2], "       ", 0 , "  " , 1 , "  " , 2)
        print("     ", b[3] , "  " , b[4] , "  " , b[5], "  ->   ", 3 , "  " , 4 , "  " , 5)
        print("     ", b[6] , "  " , b[7] , "  " , b[8], "       ", 6 , "  " , 7 , "  " , 8)
        print()

### Execution

Finally we have to implement our **main** function to play with our MCTS Artificial player

In [ ]:
import graphviz as gv

playerIA = 2
jeu = oxo()
mcts = mcts3(jeu, playerIA)

In [ ]:
# n est le nombre de partie faites par MCTS pour construire son arbre à chaque coup
# IAplayer est 1 ou 2 sachant que1 commence toujours
def jouer(n, IAplayer):
    playerIA = IAplayer
    jeu = oxo()
    mcts = mcts3(jeu, playerIA)
    loop = True
    while (loop):
        jeu = oxo()
        while jeu.end_game == 0:    
            current_state = jeu.board
            current_player = jeu.joueur(current_state)
            b = current_player
            # des X et des O au lieu de 1 et 2
            if b == 1: b='X'
            else: b = 'O'
        
            if current_player == playerIA: 
                action = mcts.play(current_state, n)
                #print("MCTS Action = ", action, " from ", jeu.actions)
            else:
                #action = mcts.play(current_state)
                jeu.myPrint()
                print ("action ", jeu.actions, " current_player = ", b)
                action = int(input("Player %s: " % (current_player)))
        
            # quit the game
            if action == 10:
                loop=False
                break 
            
            # change player
            if action == 99:
                if playerIA==1:
                    playerIA=2
                else:
                    playerIA=1
            
            if action in jeu.actions: 
                jeu.play(current_state, action)
            else: 
                print ("----- > Wrong move, try again!")
            
                
            if jeu.asWin():
                if current_player == playerIA:
                    print("------------------------------")
                    print("----------> AI WIN <----------")
                    print("------------------------------")
                else:
                    print("------------------------------")
                    print("----------> YOU WIN <---------")
                    print("------------------------------")
                jeu.end_game = current_player
        
            if jeu.nb_move == 9 and jeu.end_game == 0:
                print("------------------------------")
                print("---> No winner, No looser <---")
                print("------------------------------")
                jeu.end_game = -1
    
    # draw and export MCTS graph
    def nb_non_zero(state):
        i=0
        for s in state:
            if s!=0:
                i=i+1
        return i
    
    dot = gv.Graph(format='png')
    i=0
    stateDot = {}
    for state in mcts.graphe:
        stateDot[state]=i
        i+=1
    for state in mcts.graphe:
        dot.node(str(stateDot[state]))#, str(nb_non_zero(state)))
        for fils in mcts.graphe[state]:
            dot.edge(str(stateDot[state]), str(stateDot[fils[1]]))#, str(fils[0]))
    filename = dot.render(filename='g1')
    print(filename)
    dot.render('graphe.gv', view=True)

In [ ]:
jouer(15, 2)

## interesting behaviors

low n -> many mistake -> easy to observe that MCTS learn by experience
medium n -> already powerfull, still some mistake
high n -> always win (or nul match)

asymetric search space
able to play even for unknow state (building tree like a forest (ie. able to start from new node)

inverse reward value work very well
inverse player work well also (the tree could be used for both player)

IA vs IA: converge to match nul always




